In [ ]:
# %% [markdown]
# # Strategic Workforce Analysis: AI Integration vs. Structural Risk (2010-2025)
# ## Task 1: Data Preprocessing & Outlier Handling

# %%
import pandas as pd
import numpy as np
import os

# --- Local Path Configuration ---
BASE_DIR = os.path.dirname(os.getcwd())
RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', 'raw', 'ai_impact_jobs_2010_2025.csv')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')

# %%
# 1. Load Dataset
if os.path.exists(RAW_DATA_PATH):
    df = pd.read_csv(RAW_DATA_PATH)
    print(f"Raw Data Loaded from local. Shape: {df.shape}")
else:
    print(f"Error: File not found at {RAW_DATA_PATH}")

# 2. NESTED IQR LOGIC
# We clean outliers within each (Region + Seniority) bucket to reveal true market rates.
def get_clean_market_data(group):
    if len(group) < 3: return group # Skip tiny groups
    q1, q3 = group['salary_usd'].quantile([0.25, 0.75])
    iqr = q3 - q1
    return group[(group['salary_usd'] >= q1 - 1.5 * iqr) & (group['salary_usd'] <= q3 + 1.5 * iqr)]

# 3. Apply double-grouping for maximum precision
df_cleaned = df.groupby(['region', 'seniority_level'], group_keys=False).apply(get_clean_market_data)

# 5. Final check: Ensure 'region' and 'seniority_level' are still columns
if df_cleaned.index.name is not None or isinstance(df_cleaned.index, pd.MultiIndex):
    df_cleaned = df_cleaned.reset_index()

# %%
# 6. Save the Cleaned Dataset for subsequent tasks
# We save this to ensure consistency across all analysis branches
os.makedirs(PROCESSED_DIR, exist_ok=True)

CLEANED_FILE_PATH = os.path.join(PROCESSED_DIR, 'ai_impact_jobs_cleaned.csv')
df_cleaned.to_csv(CLEANED_FILE_PATH, index=False)

print(f"Grouped Cleaning Complete. Shape: {df_cleaned.shape}")

Raw Data Loaded from local. Shape: (5000, 22)
Precision Cleaning Results:
- Original Records: 5000
- Records after Nested IQR: 4938
- Rows removed as anomalies: 62
Grouped Cleaning Complete. Shape: (4938, 20)
